In [1]:
import os
import time
import pandas as pd
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def get_acum(pm1, pm2, Temp_df):
    
    Temp_df['PM1'] = Temp_df['Average'].rolling(pm1).mean()
    Temp_df['PM2'] = Temp_df['Average'].rolling(pm2).mean()
    Temp_df.dropna(inplace = True)
    Temp_df['Posicion'] = np.where(Temp_df['PM1'] > Temp_df['PM2'], 1, -1)
    Temp_df['Retornos'] = np.log(Temp_df['Average']/Temp_df['Average'].shift(1))
    Temp_df.dropna(inplace = True)
    Temp_df['Estrategia'] = Temp_df['Retornos']*Temp_df['Posicion'].shift(1)
    Temp_df.dropna(inplace = True)
    # Temp_df['Retacum'] = Temp_df['Retornos'].cumsum().apply(np.exp)
    Temp_df['Estracum'] = Temp_df['Estrategia'].cumsum().apply(np.exp)
    
    return Temp_df['Estracum'].tail(1).copy()

In [3]:
def best_strategy(pm1, pm2, Bitso):
    
    Bitso['PM1'] = Bitso['Average'].rolling(pm1).mean()
    Bitso['PM2'] = Bitso['Average'].rolling(pm2).mean()
    Bitso.dropna(inplace = True)
    Bitso.plot(figsize=(12,8))
    Bitso['Posicion'] = np.where(Bitso['PM1'] > Bitso['PM2'], 1, -1)
    Bitso.plot(figsize=(12,8), secondary_y = 'Posicion')
    Bitso['Retornos'] = np.log(Bitso['Average']/Bitso['Average'].shift(1))
    Bitso.dropna(inplace = True)
    Bitso['Estrategia'] = Bitso['Retornos']* Bitso['Posicion'].shift(1)
    Bitso.dropna(inplace = True)
    Bitso['Retacum'] = Bitso['Retornos'].cumsum().apply(np.exp)
    Bitso['Estracum'] = Bitso['Estrategia'].cumsum().apply(np.exp)

    #------------------Print it out
    fig = plt.figure(figsize=(14,14))

    ax1 = plt.subplot2grid((25,1),(0,0),rowspan = 8, colspan = 1)
    Bitso['Average'].plot(title = ('Estrategia PMs ' + str(pm1*10) + ', ' + str(pm2*10)))
    Bitso['PM1'].plot(ax=ax1)
    Bitso['PM2'].plot(ax=ax1)

    ax1.set_ylabel('Ultimo Precio, PM1, PM2')
    plt.xticks([],[])

    ax2 = plt.subplot2grid((25,1),(8,0), rowspan = 3, colspan = 1)
    Bitso['Posicion'].plot(ylim=[-1.1,1.1])
    ax2.set_ylabel('Posicion')
    plt.xticks([],[])

    ax3=plt.subplot2grid((25,1),(11,0),rowspan = 8, colspan = 1)
    Bitso[['Retacum', 'Estracum']].plot(ax = ax3)
    ax3.set_ylabel('Retornos Acumulados')
    plt.xticks([],[])
    
    return Bitso

In [4]:
'''
Best So Far:
PM1:
PM2:
Score:
'''

# We will go with PM1: 849 and PM2: 912

'\nBest So Far:\nPM1:\nPM2:\nScore:\n'

In [9]:
# Gross Tunning

Best_Stra = 0
Best_PM1 = 0
Best_PM2 = 0
Start_PM1 = 580  # 380
Start_PM2 = 1   # 381
Last_PM1 = 600 # 1 dia = 1440, 30 dias = 43200 min
Last_PM2 = 1440
Steps_PM1 = 1
Steps_PM2 = 2

the_path = os.getcwd()
data_path = os.path.join(the_path, '../data/processed-data/')
file_name = 'BTC-USDT-processed.csv'
file_path = data_path + file_name
df = pd.read_csv(file_path, index_col='Date')

df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df.index = pd.to_datetime(df.index)
df_M = df.last('31D').copy()
print(df_M.shape)

#fig = plt.figure()
#ax = fig.add_subplot(111)
#plt.ion()

#fig.show()
#fig.canvas.draw()
#i = 0
#L_i = []
#L_v = []

#For cycle para ver cual es la mejor combinacion

for pm1 in np.arange(Start_PM1, Last_PM1, Steps_PM1):
    for pm2 in np.arange(Start_PM2, Last_PM2, Steps_PM2):
        if (pm1<pm2) and (pm1/pm2>0.35) and ((pm1+2)<pm2):
            # i += 1
            Val_Stra = get_acum(pm1, pm2, df_M.copy())
            # L_i.append(i)
            # L_v.append(Val_Stra)
            # ax.clear()
            # ax.plot(L_i,L_v)
            # fig.canvas.draw()
            # print ('\rActual || PM1: {} | PM2: {} | Score: {} || Best || PM1: {} | PM2: {} | Score: {}'.format(pm1,pm2,Val_Stra[0],Best_PM1,Best_PM2,Best_Stra),end='')
            print ('\rActual || PM1: {} | PM2: {} | Score: {}'.format(pm1,pm2,Val_Stra[0]),end='')
            if (Val_Stra[0] > 1.4):
                print ('\n')
            if (Val_Stra[0] > Best_Stra):
                Best_Stra = Val_Stra[0]
                Best_PM1 = pm1
                Best_PM2 = pm2
print('\nBEST: {}'.format(Best_Stra))

(44640, 1)
Actual || PM1: 580 | PM2: 645 | Score: 1.4907180414837267

Actual || PM1: 580 | PM2: 647 | Score: 1.506165462958733

Actual || PM1: 580 | PM2: 649 | Score: 1.4035203841455317

Actual || PM1: 580 | PM2: 651 | Score: 1.4018116423234832

Actual || PM1: 580 | PM2: 653 | Score: 1.403647121629645

Actual || PM1: 581 | PM2: 643 | Score: 1.43230150523208568

Actual || PM1: 581 | PM2: 645 | Score: 1.4304858396110192

Actual || PM1: 581 | PM2: 647 | Score: 1.436450839668386

Actual || PM1: 581 | PM2: 651 | Score: 1.4348178947892518

Actual || PM1: 581 | PM2: 653 | Score: 1.4133273594130458

Actual || PM1: 582 | PM2: 641 | Score: 1.40447578240775522

Actual || PM1: 582 | PM2: 643 | Score: 1.425680541783774

Actual || PM1: 582 | PM2: 645 | Score: 1.4635047717572294

Actual || PM1: 582 | PM2: 647 | Score: 1.421111749348429

Actual || PM1: 582 | PM2: 649 | Score: 1.4025732854319612

Actual || PM1: 582 | PM2: 651 | Score: 1.409967435045329

Actual || PM1: 582 | PM2: 653 | Score: 1.41678107

Actual || PM1: 596 | PM2: 609 | Score: 1.45140564278600839

Actual || PM1: 596 | PM2: 621 | Score: 1.4026188685654142

Actual || PM1: 596 | PM2: 631 | Score: 1.4290835682171685

Actual || PM1: 596 | PM2: 633 | Score: 1.4316936982416364

Actual || PM1: 596 | PM2: 635 | Score: 1.4104339344749295

Actual || PM1: 596 | PM2: 637 | Score: 1.4648704711253842

Actual || PM1: 596 | PM2: 639 | Score: 1.460025615053993

Actual || PM1: 596 | PM2: 641 | Score: 1.4392515661452416

Actual || PM1: 596 | PM2: 647 | Score: 1.4079738262731768

Actual || PM1: 596 | PM2: 649 | Score: 1.4677961928090288

Actual || PM1: 596 | PM2: 651 | Score: 1.445328950862962

Actual || PM1: 596 | PM2: 653 | Score: 1.406723538389905

Actual || PM1: 596 | PM2: 655 | Score: 1.401843742317322

Actual || PM1: 596 | PM2: 665 | Score: 1.4393728054267843

Actual || PM1: 596 | PM2: 667 | Score: 1.4341487988631474

Actual || PM1: 596 | PM2: 669 | Score: 1.4003511082318107

Actual || PM1: 597 | PM2: 609 | Score: 1.43108582895698375


In [10]:
Best_PM1

595

In [11]:
Best_PM2

649